# SEGONA PART: TEXT SIMILARITY

En aquest apartat es farà una comparació d'un model amb una Xarxa Neuronal fent servir diferents tipus de word embeddings.

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
import spacy
import numpy as np
from typing import List, Tuple, Dict
import tensorflow as tf
from scipy.stats import pearsonr
from transformers import pipeline, AutoTokenizer
from scipy.special import logit
from gensim.models import TfidfModel
from gensim.utils import simple_preprocess
from gensim.corpora import Dictionary
from scipy import spatial
import pandas as pd
from gensim.models import Word2Vec
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import normalize
from datasets import load_dataset

dataset = load_dataset("projecte-aina/sts-ca")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.


In [ ]:
# Carreguem stopword en català
STOPWORDS = {"a", "abans", "abans-d'ahir", "abintestat", "ací", "adesiara", "adés", "adéu", "adàgio", "ah", "ahir", "ai",
        "aitambé", "aitampoc", "aitan", "aitant", "aitantost", "aixà", "això", "així", "aleshores", "algun", "alguna",
        "algunes", "alguns", "algú", "alhora", "allà", "allèn", "allò", "allí", "almenys", "alto", "altra", "altre",
        "altres", "altresí", "altri", "alça", "al·legro", "amargament", "amb", "ambdues", "ambdós", "amunt", "amén",
        "anc", "andante", "andantino", "anit", "ans", "antany", "apa", "aprés", "aqueix", "aqueixa", "aqueixes",
        "aqueixos", "aqueixs", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests",
        "aquèn", "aquí", "ara", "arran", "arrera", "arrere", "arreu", "arri", "arruix", "atxim", "au", "avall", "avant",
        "aviat", "avui", "açò", "bah", "baix", "baldament", "ballmanetes", "banzim-banzam", "bastant", "bastants", "ben",
        "bis", "bitllo-bitllo", "bo", "bé", "ca", "cada", "cal", "cap", "car", "caram", "catorze", "cent", "centes",
        "cents", "cerca", "cert", "certa", "certes", "certs", "cinc", "cinquanta", "cinquena", "cinquenes", "cinquens",
        "cinquè", "com", "comsevulla", "contra", "cordons", "corrents", "cric-crac", "d", "daixonses", "daixò",
        "dallonses", "dallò", "dalt", "daltabaix", "damunt", "darrera", "darrere", "davall", "davant", "de", "debades",
        "dedins", "defora", "dejorn", "dejús", "dellà", "dementre", "dempeus", "demés", "demà", "des", "desena",
        "desenes", "desens", "després", "dessobre", "dessota", "dessús", "desè", "deu", "devers", "devora", "deçà",
        "diferents", "dinou", "dins", "dintre", "disset", "divers", "diversa", "diverses", "diversos", "divuit", "doncs",
        "dos", "dotze", "dues", "durant", "ecs", "eh", "el", "ela", "elis", "ell", "ella", "elles", "ells", "els", "em",
        "emperò", "en", "enans", "enant", "encara", "encontinent", "endalt", "endarrera", "endarrere", "endavant",
        "endebades", "endemig", "endemés", "endemà", "endins", "endintre", "enfora", "enguany", "enguanyasses", "enjús",
        "enlaire", "enlloc", "enllà", "ens", "ensems", "ensota", "ensús", "entorn", "entre", "entremig", "entretant",
        "entrò", "envers", "envides", "environs", "enviró", "ençà", "ep", "ep", "era", "eren", "eres", "ergo", "es",
        "escar", "essent", "esser", "est", "esta", "estada", "estades", "estan", "estant", "estar", "estaran", "estarem",
        "estareu", "estaria", "estarien", "estaries", "estaré", "estarà", "estaràs", "estaríem", "estaríeu", "estat",
        "estats", "estava", "estaven", "estaves", "estem", "estes", "esteu", "estic", "estiguem", "estigueren",
        "estigueres", "estigues", "estiguessis", "estigueu", "estigui", "estiguin", "estiguis", "estigué", "estiguérem",
        "estiguéreu", "estigués", "estiguí", "estos", "està", "estàs", "estàvem", "estàveu", "et", "etc", "etcètera",
        "ets", "excepte", "fins", "for", "haver", "i", "inclòs", "jo", "la", "les", "llarg", "llavors", "mentre", "meu",
        "mode", "molt", "molts", "nosaltres", "o", "on", "per", "però", "perquè", "podem",
        "poden", "poder", "podeu", "potser", "primer", "puc", "quan", "quant", "que", "què",
        "qui", "sabem", "saben", "saber", "sabeu", "sap", "saps", "sense", "ser", "seu",
        "seus", "si", "soc", "solament", "sols", "som", "sota", "també", "te", "tene",
        "tenim", "tenir", "teniu", "teu", "tinc", "tot", "últim", "un", "una", "unes",
        "uns", "ús", "va", "vaig", "van", "vosaltres"}

In [ ]:
input_pairs = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["train"].to_list()]
input_pairs_val = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["validation"].to_list()]
input_pairs_test = [(e["sentence1"], e["sentence2"], e["label"], ) for e in dataset["test"].to_list()]

# Funció de preprocessament
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS]
    return preprocessed

all_input_pairs = input_pairs + input_pairs_val + input_pairs_test

sentences_1_preproc = [preprocess(sentence_1) for sentence_1, _, _ in all_input_pairs]
sentences_2_preproc = [preprocess(sentence_2) for _, sentence_2, _ in all_input_pairs]
sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))

sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc

Guardarem tots els resultats dels nostres models en una taula

In [ ]:
resultats_models = pd.DataFrame(columns=['Model Name', 'Pearson_Train', 'Pearson_Val', 'Pearson_Test'])

In [ ]:
def afegir_resultats(model_name, Pearson_Train, Pearson_Val, Pearson_Test, dataframe):

    """
    Donat el nom d'un model i els seus resultats, els afegeix al DataFrame on es
    guarda la comparació de resultats.
    """

    new_row = pd.DataFrame({
        'Model Name': [model_name],
        'Pearson_Train': [Pearson_Train],
        'Pearson_Val': [Pearson_Val],
        'Pearson_Test': [Pearson_Test],
    })

    updated_dataframe = pd.concat([dataframe, new_row], ignore_index=True)
    return updated_dataframe

In [ ]:
def build_and_compile_model(embedding_size: int = 13125, learning_rate: float = 1e-3) -> tf.keras.Model:
    """
    Funció que s'encarrega de crear la Xarxa Neuronal i compilar-la.
    """
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    first_projection = tf.keras.layers.Dense(
        embedding_size,
        kernel_initializer=tf.keras.initializers.Identity(),
        bias_initializer=tf.keras.initializers.Zeros(),
    )
    projected_1 = first_projection(input_1)
    projected_2 = first_projection(input_2)

    def cosine_distance(x):
        x1, x2 = x
        x1_normalized = tf.keras.backend.l2_normalize(x1, axis=1)
        x2_normalized = tf.keras.backend.l2_normalize(x2, axis=1)
        return 2.5 * (1.0 + tf.reduce_sum(x1_normalized * x2_normalized, axis=1))

    output = tf.keras.layers.Lambda(cosine_distance)([projected_1, projected_2])

    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    model.compile(loss='mean_squared_error',
                  optimizer=tf.keras.optimizers.Adamax(learning_rate))
    return model

In [ ]:
def compute_pearson(x_, y_, model):
    """
    Funció que ens indica lo bo que és un model,
    donades les prediccions i els valors reals,
    calcula la correlació de pearson.
    """
    y_pred = model.predict(x_)
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

In [ ]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

In [ ]:
def entrenament_i_avaluacio_model(nom_model, funcio_map, resultats_models):
  """
  Funció que mapeja les dades de la manera corresponent donada la funció que li diguem (spacy, roberta...)
  Després entrena el model i calcula la correlació de Pearson
  """

  mapped = funcio_map(all_input_pairs)
  mapped_train = funcio_map(input_pairs)
  mapped_val = funcio_map(input_pairs_val)
  mapped_test = funcio_map(input_pairs_test)

  x_train, y_train = pair_list_to_x_y(mapped_train)
  x_val, y_val = pair_list_to_x_y(mapped_val)
  x_test, y_test = pair_list_to_x_y(mapped_test)

  batch_size: int = 528
  num_epochs: int = 64

  train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
  train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

  val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
  val_dataset = val_dataset.batch(batch_size)

  test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
  test_dataset = test_dataset.batch(batch_size)

  for vectors, similitud in mapped[:5]:
      print(f"Parells de vectors: {vectors[0].shape}, {vectors[1].shape}")
      print(f"Puntuació de similitud: {similitud}")

  model = build_and_compile_model(int(vectors[0].shape[0]))
  print(model.summary())

  model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)
  print(f"RESULTATS {nom_model}")
  print(f"Correlació de Pearson (train): {compute_pearson(x_train, y_train, model)}")
  print(f"Correlació de Pearson (validation): {compute_pearson(x_val, y_val, model)}")
  print(f"Correlació de Pearson (test): {compute_pearson(x_test, y_test, model)}")

  resultats_models = afegir_resultats(nom_model, compute_pearson(x_train, y_train, model), compute_pearson(x_val, y_val, model),compute_pearson(x_test, y_test, model), resultats_models)
  return resultats_models


## 1. ONE HOT ENCODING

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from collections import Counter

word_freq = Counter(word for sentence in sentences_pairs_flattened for word in sentence)

filtered_words = [word for word, freq in word_freq.items() if freq > 10]


def preprocess(sentence: str) -> List[str]:
    return sentence.lower().split()

def map_one_hot(sentence_preproc: List[str], dictionary: Dict[str, int]) -> np.ndarray:
    # Crea un vector one-hot per la oració donada
    vector = np.zeros(len(dictionary))
    for word in sentence_preproc:
        if word in dictionary:
            index = dictionary[word]
            vector[index] = 1
    return vector

def map_pairs_one_hot(sentence_pairs: List[Tuple[str, str, float]], dictionary: Dict[str, int]) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    pares_vectores = []
    for sentence_1, sentence_2, similitud in sentence_pairs:
        sentence_1_preproc = preprocess(sentence_1)
        sentence_2_preproc = preprocess(sentence_2)
        vector1 = map_one_hot(sentence_1_preproc, dictionary)
        vector2 = map_one_hot(sentence_2_preproc, dictionary)
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

mapped = map_pairs_one_hot(all_input_pairs, word_freq)
mapped_train = map_pairs_one_hot(input_pairs, word_freq)
mapped_val = map_pairs_one_hot(input_pairs_val, word_freq)
mapped_test = map_pairs_one_hot(input_pairs_test, word_freq)
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
one_hot_encoder.fit(np.array(filtered_words).reshape(-1, 1))

batch_size = 528

x_train, y_train = pair_list_to_x_y(mapped_train)
x_val, y_val = pair_list_to_x_y(mapped_val)
x_test, y_test = pair_list_to_x_y(mapped_test)

train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)

val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.batch(batch_size)

for vectors, similitud in mapped[:5]:
    print(f"Parells de vectores: {vectors[0].shape}, {vectors[1].shape}")
    print(f"Puntuació de similitud: {similitud}")

In [ ]:
model = build_and_compile_model(18528)
print(model.summary())

# Es defineixen les constants
batch_size: int = 256
num_epochs: int = 64

# S'entrena el model
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

def compute_pearson(x_, y_):
    y_pred = model.predict(x_)
    correlation, _ = pearsonr(y_pred.flatten(), y_.flatten())
    return correlation

print(f"Correlació de Pearson (train): {compute_pearson(x_train, y_train)}")
print(f"Correlació de Pearson (validation): {compute_pearson(x_val, y_val)}")
print(f"Correlació de Pearson (test): {compute_pearson(x_test, y_test)}")

## 2. WORD2VEC

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Es carrega el dataset
dataset_ts = load_dataset("projecte-aina/sts-ca")
train_data = dataset_ts['train']
val_data = dataset_ts['validation']
test_data = dataset_ts['test']

train_data = [(preprocess(s1), preprocess(s2), label) for s1, s2, label in zip(train_data['sentence1'], train_data['sentence2'], train_data['label'])]
val_data = [(preprocess(s1), preprocess(s2), label) for s1, s2, label in zip(val_data['sentence1'], val_data['sentence2'], val_data['label'])]
test_data = [(preprocess(s1), preprocess(s2), label) for s1, s2, label in zip(test_data['sentence1'], test_data['sentence2'], test_data['label'])]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:1491: FutureWarning: The repository for projecte-aina/sts-ca contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/projecte-aina/sts-ca
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [ ]:
#  Funció per a codificar les frases utilitzant Word2Vec + Mean
def word2vec_mean_encode(sentence, model):
    if isinstance(sentence, list):
        sentence = ' '.join(sentence)
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if not word_vectors:
        return np.zeros(model.vector_size)
    return np.mean(word_vectors, axis=0)

#  Funció per a codificar les frases utilitzant Word2Vec + Mean ponderada (TF-IDF)
def word2vec_tfidf_encode(sentence, model, tfidf_model):
    if isinstance(sentence, list):
        sentence = ' '.join(sentence)
    words = sentence.split()
    word_vectors = []
    weights = []
    for word in words:
        if word in model.wv and word in tfidf_model.vocabulary_:
            word_vectors.append(model.wv[word])
            weights.append(tfidf_model.idf_[tfidf_model.vocabulary_[word]])
    if not word_vectors:
        return np.zeros(model.vector_size)
    weighted_vectors = np.average(word_vectors, axis=0, weights=weights)
    return weighted_vectors

In [ ]:
# Es carrega el model de Word2Vec preentrenat de 1GB
WORD_EMBEDDING_FILE = '/content/drive/MyDrive/Pràctica 4 PLH/models Word2Vec/catalan_word2vec_1GB.model'
word2vec_model = Word2Vec.load(WORD_EMBEDDING_FILE)
vector_size = word2vec_model.vector_size

In [ ]:
def preprocess_sentence(sentence):
    if isinstance(sentence, list):
        return ' '.join(sentence)
    return sentence

# Es prepara el corpus per a TF-IDF
corpus = [" ".join([preprocess_sentence(s1), preprocess_sentence(s2)]) for s1, s2, _ in train_data]

# Es crea i s'ajusta el model TF-IDF
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(corpus)

# Es preparen les dades d'entrenament, validació i prova utilitzant Word2Vec + Mean
X_train_1_mean = np.array([word2vec_mean_encode(preprocess_sentence(s1), word2vec_model) for s1, _, _ in train_data])
X_train_2_mean = np.array([word2vec_mean_encode(preprocess_sentence(s2), word2vec_model) for _, s2, _ in train_data])
X_val_1_mean = np.array([word2vec_mean_encode(preprocess_sentence(s1), word2vec_model) for s1, _, _ in val_data])
X_val_2_mean = np.array([word2vec_mean_encode(preprocess_sentence(s2), word2vec_model) for _, s2, _ in val_data])
X_test_1_mean = np.array([word2vec_mean_encode(preprocess_sentence(s1), word2vec_model) for s1, _, _ in test_data])
X_test_2_mean = np.array([word2vec_mean_encode(preprocess_sentence(s2), word2vec_model) for _, s2, _ in test_data])

# Es preparen les dades d'entrenament, validació i prova utilitzant Word2Vec + TF-IDF
X_train_1_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s1), word2vec_model, tfidf_vectorizer) for s1, _, _ in train_data])
X_train_2_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s2), word2vec_model, tfidf_vectorizer) for _, s2, _ in train_data])
X_val_1_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s1), word2vec_model, tfidf_vectorizer) for s1, _, _ in val_data])
X_val_2_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s2), word2vec_model, tfidf_vectorizer) for _, s2, _ in val_data])
X_test_1_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s1), word2vec_model, tfidf_vectorizer) for s1, _, _ in test_data])
X_test_2_tfidf = np.array([word2vec_tfidf_encode(preprocess_sentence(s2), word2vec_model, tfidf_vectorizer) for _, s2, _ in test_data])

# S'extreuen les etiquetes dels conjunts d'entrenament, validació i prova
y_train = np.array([label for _, _, label in train_data])
y_val = np.array([label for _, _, label in val_data])
y_test = np.array([label for _, _, label in test_data])

### 2.1 WORD2VEC + Mean

In [ ]:
# Es construeix i compila el model
model_mean = build_and_compile_model(embedding_size=word2vec_model.vector_size)

# S'entrena el model utilitzant Word2Vec + Mean
model_mean.fit([X_train_1_mean, X_train_2_mean], y_train, epochs=10, batch_size=32)

# S'evalua el model utilitzant Word2Vec + Mean
print("Correlació de Pearson (Word2Vec + Mean) - train:", compute_pearson([X_train_1_mean, X_train_2_mean], y_train, model_mean))
print("Correlació de Pearson (Word2Vec + Mean) - validation:", compute_pearson([X_val_1_mean, X_val_2_mean], y_val, model_mean))
print("Correlació de Pearson (Word2Vec + Mean) - test:", compute_pearson([X_test_1_mean, X_test_2_mean], y_test, model_mean))
resultats_models = afegir_resultats("Word2Vec + Mean",compute_pearson([X_train_1_mean, X_train_2_mean], y_train, model_mean), compute_pearson([X_val_1_mean, X_val_2_mean], y_val, model_mean), compute_pearson([X_test_1_mean, X_test_2_mean], y_test, model_mean), resultats_models)

Epoch 1/10
65/65 [==============================] - 4s 9ms/step - loss: 4.2928
Epoch 2/10
65/65 [==============================] - 1s 8ms/step - loss: 3.8653
Epoch 3/10
65/65 [==============================] - 1s 9ms/step - loss: 3.5913
Epoch 4/10
65/65 [==============================] - 1s 8ms/step - loss: 3.4053
Epoch 5/10
65/65 [==============================] - 0s 7ms/step - loss: 3.2640
Epoch 6/10
65/65 [==============================] - 0s 7ms/step - loss: 3.1490
Epoch 7/10
65/65 [==============================] - 1s 9ms/step - loss: 3.0539
Epoch 8/10
65/65 [==============================] - 0s 4ms/step - loss: 2.9729
Epoch 9/10
65/65 [==============================] - 0s 7ms/step - loss: 2.9024
Epoch 10/10
65/65 [==============================] - 1s 9ms/step
Correlación de Pearson (Word2Vec + Mean) - train: 0.4680761382152332
16/16 [==============================] - 0s 10ms/step
Correlación de Pearson (Word2Vec + Mean) - validation: 0.34795918571490075
16/16 [===================

### 2.2 WORD2VEC + TF-IDF

In [ ]:
# Es construeix i compila el model
model_tfidf = build_and_compile_model(embedding_size=word2vec_model.vector_size)

# S'entrena el model utilitzant Word2Vec + TF-IDF
model_tfidf.fit([X_train_1_tfidf, X_train_2_tfidf], y_train, epochs=10, batch_size=32)

# S'evalua el model utilitzant Word2Vec + TF-IDF
print("Correlació de Pearson (Word2Vec + TF-IDF) - train:", compute_pearson([X_train_1_tfidf, X_train_2_tfidf], y_train, model_tfidf))
print("Correlació de Pearson (Word2Vec + TF-IDF) - validation:", compute_pearson([X_val_1_tfidf, X_val_2_tfidf], y_val, model_tfidf))
print("Correlació de Pearson (Word2Vec + TF-IDF) - test:", compute_pearson([X_test_1_tfidf, X_test_2_tfidf], y_test, model_tfidf))

resultats_models = afegir_resultats("Word2Vec + TF-IDF", compute_pearson([X_train_1_tfidf, X_train_2_tfidf], y_train, model_tfidf), compute_pearson([X_val_1_tfidf, X_val_2_tfidf], y_val, model_tfidf), compute_pearson([X_test_1_tfidf, X_test_2_tfidf], y_test, model_tfidf), resultats_models)

Epoch 1/10
65/65 [==============================] - 2s 7ms/step - loss: 4.3834
Epoch 2/10
65/65 [==============================] - 1s 8ms/step - loss: 3.9665
Epoch 3/10
65/65 [==============================] - 0s 5ms/step - loss: 3.6915
Epoch 4/10
65/65 [==============================] - 0s 4ms/step - loss: 3.5014
Epoch 5/10
65/65 [==============================] - 0s 5ms/step - loss: 3.3522
Epoch 6/10
65/65 [==============================] - 0s 4ms/step - loss: 3.2290
Epoch 7/10
65/65 [==============================] - 0s 6ms/step - loss: 3.1246
Epoch 8/10
65/65 [==============================] - 0s 5ms/step - loss: 3.0341
Epoch 9/10
65/65 [==============================] - 0s 4ms/step - loss: 2.9530
Epoch 10/10
65/65 [==============================] - 0s 4ms/step
Correlación de Pearson (Word2Vec + TF-IDF) - train: 0.5183954338883123
16/16 [==============================] - 0s 5ms/step
Correlación de Pearson (Word2Vec + TF-IDF) - validation: 0.3716283430396442
16/16 [=================

In [ ]:
resultats_models

,Model Name,Pearson_Train,Pearson_Val,Pearson_Test
0,Word2Vec + Mean,0.468076,0.347959,0.411652
1,Word2Vec + TF-IDF,0.518395,0.371628,0.404743


## 3. SPACY

In [ ]:
!python3 -m spacy download ca_core_news_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 MB 11.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import spacy
nlp = spacy.load("ca_core_news_md")

In [ ]:
def text_to_spacy_vector(sentence: str) -> np.ndarray:
    doc = nlp(sentence)
    vectors = [token.vector for token in doc if token.has_vector]

    if vectors:
        mean_vector = np.mean(vectors, axis=0)
    else:
        mean_vector = np.zeros(nlp.vocab.vectors_length)
    return mean_vector

def map_pairs_spacy(sentence_pairs: List[Tuple[str, str, float]]) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    pares_vectores = []
    for sentence_1, sentence_2, similitud in sentence_pairs:
        vector1 = text_to_spacy_vector(sentence_1)
        vector2 = text_to_spacy_vector(sentence_2)
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [ ]:
resultats_models = entrenament_i_avaluacio_model("Spacy", map_pairs_spacy, resultats_models)

Parells de vectors: (300,), (300,)
Puntuació de similitud: 3.5
Parells de vectors: (300,), (300,)
Puntuació de similitud: 1.25
Parells de vectors: (300,), (300,)
Puntuació de similitud: 3.6700000762939453
Parells de vectors: (300,), (300,)
Puntuació de similitud: 2.25
Parells de vectors: (300,), (300,)
Puntuació de similitud: 2.0
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 300)]                0         []                            
                                                                                                  
 dense_2 (Dense)             (None, 300)                 

In [ ]:
resultats_models

,Model Name,Pearson_Train,Pearson_Val,Pearson_Test
0,Word2Vec + Mean,0.468076,0.347959,0.411652
1,Word2Vec + TF-IDF,0.518395,0.371628,0.404743
2,Spacy,0.522912,0.312013,0.430223


## 4. ROBERTA

In [ ]:
!python -m spacy download ca_core_news_trf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 MB 1.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ca_core_news_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# S'importa el Roberta
nlp = spacy.load("ca_core_news_trf")

### 4.1 ROBERTA CLS

In [ ]:
def text_to_roberta_cls(sentence: str) -> np.ndarray:
    doc = nlp(sentence)
    new_doc = doc._.trf_data.last_hidden_layer_state.dataXd[0]
    return new_doc

def map_pairs_roberta_cls(sentence_pairs: List[Tuple[str, str, float]]) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    pares_vectores = []
    for sentence_1, sentence_2, similitud in sentence_pairs:
        vector1 = text_to_roberta_cls(sentence_1)
        vector2 = text_to_roberta_cls(sentence_2)
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [ ]:
resultats_models = entrenament_i_avaluacio_model("Roberta CLS", map_pairs_roberta_cls, resultats_models)

Parells de vectors: (768,), (768,)
Puntuació de similitud: 3.5
Parells de vectors: (768,), (768,)
Puntuació de similitud: 1.25
Parells de vectors: (768,), (768,)
Puntuació de similitud: 3.6700000762939453
Parells de vectors: (768,), (768,)
Puntuació de similitud: 2.25
Parells de vectors: (768,), (768,)
Puntuació de similitud: 2.0
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_7 (InputLayer)        [(None, 768)]                0         []                            
                                                                                                  
 input_8 (InputLayer)        [(None, 768)]                0         []                            
                                                                                                  
 dense_3 (Dense)             (None, 768)                 

In [ ]:
resultats_models

,Model Name,Pearson_Train,Pearson_Val,Pearson_Test
0,Word2Vec + Mean,0.468076,0.347959,0.411652
1,Word2Vec + TF-IDF,0.518395,0.371628,0.404743
2,Spacy,0.522912,0.312013,0.430223
3,Roberta CLS,0.625978,0.183531,0.335894


### 4.2 ROBERTA MEAN

In [ ]:
def text_to_roberta_mean(sentence: str) -> np.ndarray:
    doc = nlp(sentence)
    new_doc = doc._.trf_data.last_hidden_layer_state.dataXd.mean(axis=0)
    return new_doc

def map_pairs_roberta_mean(sentence_pairs: List[Tuple[str, str, float]]) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    pares_vectores = []
    for sentence_1, sentence_2, similitud in sentence_pairs:
        vector1 = text_to_roberta_mean(sentence_1)
        vector2 = text_to_roberta_mean(sentence_2)
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [ ]:
resultats_models = entrenament_i_avaluacio_model("Roberta MEAN", map_pairs_roberta_mean, resultats_models)

Parells de vectors: (768,), (768,)
Puntuació de similitud: 3.5
Parells de vectors: (768,), (768,)
Puntuació de similitud: 1.25
Parells de vectors: (768,), (768,)
Puntuació de similitud: 3.6700000762939453
Parells de vectors: (768,), (768,)
Puntuació de similitud: 2.25
Parells de vectors: (768,), (768,)
Puntuació de similitud: 2.0
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, 768)]                0         []                            
                                                                                                  
 input_10 (InputLayer)       [(None, 768)]                0         []                            
                                                                                                  
 dense_4 (Dense)             (None, 768)                 

In [ ]:
resultats_models

,Model Name,Pearson_Train,Pearson_Val,Pearson_Test
0,Word2Vec + Mean,0.468076,0.347959,0.411652
1,Word2Vec + TF-IDF,0.518395,0.371628,0.404743
2,Spacy,0.522912,0.312013,0.430223
3,Roberta CLS,0.625978,0.183531,0.335894
4,Roberta MEAN,0.732696,0.369659,0.513428


## 5. ROBERTA FINE-TUNED

In [ ]:
model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = [(e["sentence1"], e["sentence2"], ) for e in dataset["test"].to_list()]

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    prediction['score'] = logit(prediction['score'])
print(predictions)

tokenizer_config.json:   0%|          | 0.00/1.13k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/848k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/506k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

[{'label': 'SIMILARITY', 'score': 2.1410346445880513}, {'label': 'SIMILARITY', 'score': 3.0859965063408987}, {'label': 'SIMILARITY', 'score': 1.6586843002953948}, {'label': 'SIMILARITY', 'score': 3.2563704928342947}, {'label': 'SIMILARITY', 'score': 2.4166345236177897}, {'label': 'SIMILARITY', 'score': 3.4185395395722145}, {'label': 'SIMILARITY', 'score': 2.838414066682235}, {'label': 'SIMILARITY', 'score': 3.782994356337281}, {'label': 'SIMILARITY', 'score': 2.4721741603835476}, {'label': 'SIMILARITY', 'score': 2.158547000649474}, {'label': 'SIMILARITY', 'score': 1.6486324447085778}, {'label': 'SIMILARITY', 'score': 2.378597435771381}, {'label': 'SIMILARITY', 'score': 2.6842834808452993}, {'label': 'SIMILARITY', 'score': 3.0824030060422962}, {'label': 'SIMILARITY', 'score': 1.9718790901504786}, {'label': 'SIMILARITY', 'score': 3.6191208867502}, {'label': 'SIMILARITY', 'score': 2.4404707091749747}, {'label': 'SIMILARITY', 'score': 1.9142290836862115}, {'label': 'SIMILARITY', 'score': 1

In [ ]:
sentence_pairs = [(e["sentence1"], e["sentence2"], ) for e in dataset["train"].to_list()]

predictions_train = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions_train:
    prediction['score'] = logit(prediction['score'])
print(predictions_train)

[{'label': 'SIMILARITY', 'score': 3.9050083169777956}, {'label': 'SIMILARITY', 'score': 1.2938882139528545}, {'label': 'SIMILARITY', 'score': 4.201401910346833}, {'label': 'SIMILARITY', 'score': 2.7064105493285155}, {'label': 'SIMILARITY', 'score': 1.654348056302692}, {'label': 'SIMILARITY', 'score': 2.850589694845317}, {'label': 'SIMILARITY', 'score': 3.0933511866798256}, {'label': 'SIMILARITY', 'score': 2.7604804476504947}, {'label': 'SIMILARITY', 'score': 2.5479786848723025}, {'label': 'SIMILARITY', 'score': 3.131580894689497}, {'label': 'SIMILARITY', 'score': 3.1964559619397312}, {'label': 'SIMILARITY', 'score': 0.548329756041373}, {'label': 'SIMILARITY', 'score': 1.9513812863684}, {'label': 'SIMILARITY', 'score': 3.5313567310954768}, {'label': 'SIMILARITY', 'score': 3.027013568262699}, {'label': 'SIMILARITY', 'score': 2.890897104232176}, {'label': 'SIMILARITY', 'score': 1.352541099739381}, {'label': 'SIMILARITY', 'score': 3.247333131821523}, {'label': 'SIMILARITY', 'score': 1.8342

In [ ]:
sentence_pairs = [(e["sentence1"], e["sentence2"], ) for e in dataset["validation"].to_list()]

predictions_val = pipe(prepare(sentence_pairs), add_special_tokens=False)

# convert back to scores to the original 0 and 5 interval
for prediction in predictions_val:
    prediction['score'] = logit(prediction['score'])
print(predictions_val)

[{'label': 'SIMILARITY', 'score': 1.599718914812369}, {'label': 'SIMILARITY', 'score': 2.9285647538900967}, {'label': 'SIMILARITY', 'score': 2.9183099768925826}, {'label': 'SIMILARITY', 'score': 2.419896475469414}, {'label': 'SIMILARITY', 'score': 2.565388497083661}, {'label': 'SIMILARITY', 'score': 2.966213946860535}, {'label': 'SIMILARITY', 'score': 1.8261404549684925}, {'label': 'SIMILARITY', 'score': 2.84252096456885}, {'label': 'SIMILARITY', 'score': 2.912887167453469}, {'label': 'SIMILARITY', 'score': 2.6939036464975885}, {'label': 'SIMILARITY', 'score': 1.820840340433099}, {'label': 'SIMILARITY', 'score': 2.6645098923140718}, {'label': 'SIMILARITY', 'score': 1.935774391139837}, {'label': 'SIMILARITY', 'score': 3.135998370016381}, {'label': 'SIMILARITY', 'score': 2.8623903698886486}, {'label': 'SIMILARITY', 'score': 1.1104033407345353}, {'label': 'SIMILARITY', 'score': 2.6078619026269467}, {'label': 'SIMILARITY', 'score': 2.223809771668301}, {'label': 'SIMILARITY', 'score': 4.226

In [ ]:
def compute_pearson_fine_tuned(true, pred):
    correlation, _ = pearsonr(pred, true)
    return correlation

y_true_test = [e["label"] for e in dataset["test"].to_list()]
y_pred_test = [dic['score'] for dic in predictions]

print(f"Correlació de Pearson (test): {compute_pearson_fine_tuned(y_true_test, y_pred_test)}")

Correlació de Pearson (test): 0.7819886604536139


In [ ]:
y_true_val = [e["label"] for e in dataset["validation"].to_list()]
y_pred_val = [dic['score'] for dic in predictions_val]

print(f"Correlació de Pearson (validation): {compute_pearson_fine_tuned(y_true_val, y_pred_val)}")

Correlació de Pearson (validation): 0.7522604427713253


In [ ]:
y_true_train = [e["label"] for e in dataset["train"].to_list()]
y_pred_train = [dic['score'] for dic in predictions_train]


print(f"Correlació de Pearson (train): {compute_pearson_fine_tuned(y_true_train, y_pred_train)}")

Correlació de Pearson (train): 0.9474290532358783


In [ ]:
resultats_models = afegir_resultats('Roberta fine-tuned', compute_pearson_fine_tuned(y_true_test, y_pred_test), compute_pearson_fine_tuned(y_true_val, y_pred_val), compute_pearson_fine_tuned(y_true_train, y_pred_train), resultats_models)

In [ ]:
resultats_models

,Model Name,Pearson_Train,Pearson_Val,Pearson_Test
0,Word2Vec + Mean,0.468076,0.347959,0.411652
1,Word2Vec + TF-IDF,0.518395,0.371628,0.404743
2,Spacy,0.522912,0.312013,0.430223
3,Roberta CLS,0.625978,0.183531,0.335894
4,Roberta MEAN,0.732696,0.369659,0.513428
5,Roberta fine-tuned,0.781989,0.752260,0.947429
